# Inizializzazione università e SCA

In [ ]:
from SmartContractAuthority.sca import SmartContractAuthority
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from University.university import University
from Student.student import Student
import base64
import sys

SCA_PATH = './smart_contract_authority/'
UNIVERSITIES_PATH = './universities/'
STUDENTS_PATH = './students/'
CONTRACTS_BUILD_PATH = './SmartContracts/build'

## Creazione Smart Contract Authority
Viene associato un account eth alla Smart Contract Authority che fa il deploy dei propri contratti

In [ ]:
SmartContractAuthority.create_sca(SCA_PATH, CONTRACTS_BUILD_PATH)

### Creazione dell'istanza
Viene effettivamente caricata l'istanza della SCA a partire dal file JSON di persistenza generato nel blocco precedente

In [ ]:
sca = SmartContractAuthority(SCA_PATH+"SCA", CONTRACTS_BUILD_PATH)

## Creazione Università
In questo blocco vengono create le università della rete assegnando loro un nome, che poi risulterà nelle directory dedicate, ed un account eth tramite il quale faranno il deploy dei contratti per la gestione di credenziali e studenti.

In [ ]:
University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)

University.create_university(UNIVERSITIES_PATH, CONTRACTS_BUILD_PATH)

### Creazione dell'istanza
Viene effettivamente caricata l'istanza delle università a partire dal file JSON di persistenza generato nel blocco precedente

In [ ]:
uni_origin = University(UNIVERSITIES_PATH+'University_Università Degli Studi Di Salerno', CONTRACTS_BUILD_PATH)

uni_erasmus = University(UNIVERSITIES_PATH+'University_UNICAEN', CONTRACTS_BUILD_PATH)

## Creazione studente

In [ ]:
Student.create_student(STUDENTS_PATH)

In [ ]:
studente = Student(STUDENTS_PATH+'Student_AntonioCarbone')

## Accreditamento università create

In [ ]:
UID, sca_contract_address = sca.register_university(uni_origin)
uni_origin.update_uid(UID)
uni_origin.update_sca_contract_address(sca_contract_address)

UID, sca_contract_address = sca.register_university(uni_erasmus)
uni_erasmus.update_uid(UID)
uni_erasmus.update_sca_contract_address(sca_contract_address)

## Immatricolazione presso università di origine

In [ ]:
credenziale_studente,sid = uni_origin.register_student(studente)
studente.set_sid(sid)
studente.save_credential(credenziale_studente)

## Immatricolazione presso università estera

In [ ]:
uni_erasmus.register_erasmus_student(studente, studente.credentials[0].toJSON())
uni_erasmus.update_erasmus_students(studente, credenziale_studente.CID)

## Richiesta e rilascio credenziale carriera

In [ ]:
career_credential = uni_erasmus.request_career_credential(studente)
studente.save_credential(career_credential)

## Costruzione della credenziale parziale

In [ ]:
partial_credential = studente.build_shared_credential()

studente.save_credential(partial_credential)

## Condivisione credenziale

In [ ]:
uni_origin.validate_shared_credential(studente, studente.credentials[0].toJSON())

## Revoca credenziale di immatricolazione

In [ ]:
uni_origin.revoke_cid("CID:UNISA:1")

## Revoca credenziale carriera

In [ ]:
uni_erasmus.revoke_cid("CID:ENSICAEN:0")

## Manomissione di una credenziale

## Ente certificatore fasullo

## Ente certificatore malintenzionato

## Sosia

## Forgiatore di credenziali